In [1]:
!git clone https://github.com/chop4687/SSD.git

fatal: destination path 'SSD' already exists and is not an empty directory.


In [2]:
!nvidia-smi

Tue Dec 17 08:44:09 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from tqdm import tqdm_notebook as tqdm

In [4]:
from torchvision import datasets
datasets.VOCDetection(root='SSD/', year='2007', download=True)
datasets.VOCDetection(root='SSD/', year='2012', download=True)

Using downloaded and verified file: SSD/VOCtrainval_06-Nov-2007.tar
Using downloaded and verified file: SSD/VOCtrainval_11-May-2012.tar


Dataset VOCDetection
    Number of datapoints: 5717
    Root location: SSD/

In [0]:
import sys
import warnings
warnings.filterwarnings("ignore")
sys.path.append('/content/SSD')

In [0]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import SSD
import os
import warnings
warnings.filterwarnings("ignore")
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse
from torch.nn.parallel.data_parallel import DataParallel
from torchvision import models

In [0]:
def multibox(num_classes):
    vgg = []
    # conv2d = nn.Conv2d(3, 64, kernel_size=3, padding=1)
    # vgg += [conv2d, nn.ReLU(inplace=True)]
    # conv2d = nn.Conv2d(64, 64, kernel_size=3, padding=1)
    # vgg += [conv2d, nn.ReLU(inplace=True)]
    # vgg += [nn.MaxPool2d(kernel_size=2, stride=2)]
    # conv2d = nn.Conv2d(64, 128, kernel_size=3, padding=1)
    # vgg += [conv2d, nn.ReLU(inplace=True)]
    # conv2d = nn.Conv2d(128, 128, kernel_size=3, padding=1)
    # vgg += [conv2d, nn.ReLU(inplace=True)]
    # vgg += [nn.MaxPool2d(kernel_size=2, stride=2)]
    # conv2d = nn.Conv2d(128, 256, kernel_size=3, padding=1)
    # vgg += [conv2d, nn.ReLU(inplace=True)]
    # conv2d = nn.Conv2d(256, 256, kernel_size=3, padding=1)
    # vgg += [conv2d, nn.ReLU(inplace=True)]
    # conv2d = nn.Conv2d(256, 256, kernel_size=3, padding=1)
    # vgg += [conv2d, nn.ReLU(inplace=True)]
    # vgg += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
    # conv2d = nn.Conv2d(256, 512, kernel_size=3, padding=1)
    # vgg += [conv2d, nn.ReLU(inplace=True)]
    # conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    # vgg += [conv2d, nn.ReLU(inplace=True)]
    # conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    # vgg += [conv2d, nn.ReLU(inplace=True)]
    # vgg += [nn.MaxPool2d(kernel_size=2, stride=2)]
    # conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    # vgg += [conv2d, nn.ReLU(inplace=True)]
    # conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    # vgg += [conv2d, nn.ReLU(inplace=True)]
    # conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    # vgg += [conv2d, nn.ReLU(inplace=True)]

    vgg_pretrained = models.vgg16(pretrained=True).features[:-1]
    vgg_pretrained[16] = nn.MaxPool2d(kernel_size=2, stride=2,padding=0, dilation=1 , ceil_mode=True)
    for i in range(len(vgg_pretrained)) :
        vgg += [vgg_pretrained[i]]
    vgg += [nn.MaxPool2d(kernel_size=3, stride=1,padding=1, dilation=1 , ceil_mode=False)]
    vgg += [nn.Conv2d(512,1024,kernel_size=(3,3),stride=(1,1),padding=(6,6),dilation=(6,6))]
    vgg += [nn.ReLU(inplace=True)]
    vgg += [nn.Conv2d(1024,1024,kernel_size=(1,1),stride=(1,1))]
    vgg += [nn.ReLU(inplace=True)]

    extra_layers = []
    extra_layers += [nn.Conv2d(1024, 256, kernel_size=(1, 1),stride=(1,1))]
    extra_layers += [nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(2,2), padding=(1,1))]
    extra_layers += [nn.Conv2d(512, 128, kernel_size=(1, 1),stride=(1,1))]
    extra_layers += [nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(2,2), padding=(1,1))]
    extra_layers += [nn.Conv2d(256, 128, kernel_size=(1, 1),stride=(1,1))]
    extra_layers += [nn.Conv2d(128, 256, kernel_size=(3, 3),stride=(1,1))]
    extra_layers += [nn.Conv2d(256, 128, kernel_size=(1, 1),stride=(1,1))]
    extra_layers += [nn.Conv2d(128, 256, kernel_size=(3, 3),stride=(1,1))]


    loc_layers = []
    loc_layers += [nn.Conv2d(512, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]
    loc_layers += [nn.Conv2d(1024, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]
    loc_layers += [nn.Conv2d(512, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]
    loc_layers += [nn.Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]
    loc_layers += [nn.Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]
    loc_layers += [nn.Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]

    conf_layers = []
    conf_layers += [nn.Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]
    conf_layers += [nn.Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]
    conf_layers += [nn.Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]
    conf_layers += [nn.Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]
    conf_layers += [nn.Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]
    conf_layers += [nn.Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]

    return vgg, extra_layers, (loc_layers, conf_layers)


In [0]:
def build_ssd(phase, size=300, num_classes=21):
    base_, extras_, head_ = multibox(num_classes)
    return SSD(phase, size, base_, extras_, head_, num_classes)

In [0]:
dataset = 'VOC'
dataset_root = VOC_ROOT
batch_size = 16
start_iter = 0
num_workers = 4
cuda = True
lr = 1e-4
momentum = 0.9
weight_decay = 5e-4
gamma = 0.1
save_folder = 'weights/'


In [0]:
def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def xavier(param):
    init.xavier_uniform(param)


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()

In [0]:
def train():
    cfg = voc
    dataset = VOCDetection(root=dataset_root,
                           transform=SSDAugmentation(cfg['min_dim'],
                                                     MEANS))

    ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
    net = ssd_net

    if cuda:
        net = torch.nn.DataParallel(ssd_net)
        cudnn.benchmark = True

    if cuda:
        net = net.cuda()

    print('Initializing weights...')
    # initialize newly added layers' weights with xavier method
    ssd_net.extras.apply(weights_init)
    ssd_net.loc.apply(weights_init)
    ssd_net.conf.apply(weights_init)

    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum,
                          weight_decay=weight_decay)
    criterion = MultiBoxLoss(cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,
                             False, cuda)

    net.train()
    # loss counters
    loc_loss = 0
    conf_loss = 0
    epoch = 0
    print('Loading the dataset...')

    epoch_size = len(dataset) // batch_size
    print('Training SSD on:', dataset.name)

    step_index = 0


    data_loader = data.DataLoader(dataset, batch_size,
                                  num_workers=num_workers,
                                  shuffle=True, collate_fn=detection_collate,
                                  pin_memory=True)
    # create batch iterator
    batch_iterator = iter(data_loader)
    for iteration in range(start_iter, cfg['max_iter']):
        if iteration != 0 and (iteration % epoch_size == 0):
            loc_loss = 0
            conf_loss = 0
            epoch += 1

        if iteration in cfg['lr_steps']:
            step_index += 1
            adjust_learning_rate(optimizer, gamma, step_index)

        # load train data
        try:
            images, targets = next(batch_iterator)
        except StopIteration:
            batch_iterator = iter(data_loader)
            images, targets = next(batch_iterator)
        if cuda:
            images = Variable(images.cuda())
            targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
        else:
            images = Variable(images)
            targets = [Variable(ann, volatile=True) for ann in targets]
        # forward
        t0 = time.time()
        out = net(images)
        # backprop
        optimizer.zero_grad()
        loss_l, loss_c = criterion(out, targets)
        loss = loss_l + loss_c
        loss.backward()
        optimizer.step()
        t1 = time.time()
        loc_loss += loss_l.data
        conf_loss += loss_c.data

        if iteration % 10 == 0:
            print('timer: %.4f sec.' % (t1 - t0))
            print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.data), end=' ')


        if iteration != 0 and iteration % 5000 == 0:
            print('Saving state, iter:', iteration)
            torch.save(ssd_net.state_dict(), 'weights/ssd300_' +
                       repr(iteration) + '.pth')
    torch.save(ssd_net.state_dict(),
               save_folder + '' + dataset + '.pth')

In [12]:
train()

Initializing weights...
Loading the dataset...
Training SSD on: VOC0712
timer: 9.3468 sec.
iter 0 || Loss: 60.5361 || timer: 0.1859 sec.
iter 10 || Loss: 18.9157 || timer: 0.1828 sec.
iter 20 || Loss: 15.1747 || timer: 0.2594 sec.
iter 30 || Loss: 15.2182 || timer: 0.2396 sec.
iter 40 || Loss: 14.9558 || timer: 0.1908 sec.
iter 50 || Loss: 15.2420 || timer: 0.1836 sec.
iter 60 || Loss: 14.9179 || timer: 0.1849 sec.
iter 70 || Loss: 15.1396 || timer: 0.1771 sec.
iter 80 || Loss: 14.8052 || timer: 0.2457 sec.
iter 90 || Loss: 14.9239 || timer: 0.1661 sec.
iter 100 || Loss: 14.6687 || timer: 0.1680 sec.
iter 110 || Loss: 14.4892 || timer: 0.2300 sec.
iter 120 || Loss: 14.6413 || timer: 0.1676 sec.
iter 130 || Loss: 14.5213 || timer: 0.2291 sec.
iter 140 || Loss: 13.9567 || timer: 0.2582 sec.
iter 150 || Loss: 14.5301 || timer: 0.1688 sec.
iter 160 || Loss: 13.7863 || timer: 0.1823 sec.
iter 170 || Loss: 11.8692 || timer: 0.2520 sec.
iter 180 || Loss: 13.1458 || timer: 0.2126 sec.
iter 190

KeyboardInterrupt: ignored

In [0]:
'''-----------------------------------------------------------------------------------------------'''

In [0]:
from eval import test_net

In [0]:
num_classes = len(labelmap) + 1                      # +1 for background
net = build_ssd('val', 300, num_classes)            # initialize SSD
net.load_state_dict(torch.load(trained_model))
net.eval()
print('Finished loading model!')
# load data
dataset = VOCDetection(voc_root, [('2007', set_type)],
                        BaseTransform(300, dataset_mean),
                        VOCAnnotationTransform())
if cuda:
    net = net.cuda()
    cudnn.benchmark = True
# evaluation
test_net(save_folder, net, cuda, dataset,
          BaseTransform(net.size, dataset_mean), top_k, 300,
          thresh=confidence_threshold)
